# e-stat File converter 
##### GIS_workshop_0425 版
---
- ①統計データ加工 (txt_to_xlsx)
- ②統計データ結合 (statistics_merge) ※GISでもできます
---
始める前にimportしましょう↓

In [ ]:
import pandas as pd
import glob
import os
import zipfile
#前提モジュール


### **・①txt_to_xlsx (統計データ加工コード)**

☆使い方☆<br>
① 実行する前に・・・当ipynbの直下に「txt_to_xlsx_convert」用のファイルを作りましょう。(その中のファイルを加工してくれます)<br>
② estat統計メッシュデータの「人口及び世帯」のみ対応しています<br>
③ ダウンロードしたzipファイルから、名前を変えずにconvert用のファイルに入れてください<br>
④ 下のコードを実行してください！

---
↓このセルから実行↓

In [ ]:
zip_path = "./txt_to_xlsx_convert/*zip" #パスでエラーが出たらここを変えてください
convert_path = "./txt_to_xlsx_convert"
zip_df_list = [os.path.splitext(os.path.basename(file))[0] for file in glob.glob(zip_path)]
if len(zip_df_list) == 0:
    print("zipファイルが存在しません")
else:
    print(zip_df_list) #ファイルがすべて入っているか確認しよう

#確認出来たら下のセルを実行しよう


In [ ]:
year = input("西暦何年のデータですか？")
mesh_num = input("何次メッシュですか？(メッシュでない場合は空白で)")
for file_name in zip_df_list:
    with zipfile.ZipFile(f"{convert_path}/{file_name}.zip", 'r') as zip_ref:
        zip_ref.extractall(convert_path)
    df = pd.read_csv(f"{convert_path}/{file_name}.txt",sep=',', encoding='shift_jis')
    df_nan_name_list = []
    for k in range(4):
        df_nan_name_list.append(df.columns[k]) #前4列の列名を1つ下におろす作業
    df.columns = df.iloc[0]
    df = df.drop(index=0).reset_index(drop=True)
    df = df.rename_axis(None, axis=1)
    for i in range(len(df_nan_name_list)):
        df = df.rename(columns={df.columns[i]: df_nan_name_list[i]})
    df.columns = df.columns.str.replace('　', '')
    df = df[df['HTKSYORI'] != 2] #HTKSYORIが2の列、GASSAN列を削除
    df = df.drop(columns=['GASSAN','HTKSAKI'])
    df['KEY_CODE'] = df['KEY_CODE'].astype(int)
    df['KEY_CODE'] = df['KEY_CODE'].astype(str)
    df.replace('*', 0, inplace=True)
    for column in df.columns:
        if column != 'KEY_CODE':
            df[column] = pd.to_numeric(df[column], errors='coerce') #KEY_CODE以外を数値型に変換
    df.to_excel(f'{convert_path}/{year}_{mesh_num}mesh_jinko_{file_name[-4:]}.xlsx', index=False)
    print(f"・{file_name}.xlsx を作成しました")
    os.remove(f"{convert_path}/{file_name}.zip")
    os.remove(f"{convert_path}/{file_name}.txt")
#保存したものは統計データに入れておきましょう
print("全ファイル変換完了しました！")


---

### **・②statistics_merge (統計データ結合コード)**

※実行する前に・・・当ipynbの直下に「statistics_merge」用のファイルを作りましょう。(その中のファイルを結合してくれます)<br>
※xlsxのみ読み取りに対応しています。①でtxtをあらかじめ変更しておきましょう<br>
※データは再利用できるので、極力GIS上で結合しましょう(重い場合はこちらを利用してください)

---
↓このセルから実行↓

In [ ]:
xlsx_path = "./statistics_merge/*xlsx" #パスでエラーが出たらここを変えてください
stats_path = "./statistics_merge"
xlsx_df_list = [os.path.splitext(os.path.basename(file))[0] for file in glob.glob(xlsx_path)]
if len(xlsx_df_list) <= 1:
    print("【警告】2つ以上のデータをファイルに入れてください")
else:
    print(xlsx_df_list) #ファイルがすべて入っているか確認しよう
#確認出来たら下のセルを実行しよう


In [ ]:
merge_df = pd.DataFrame()
merge_name = input("出力ファイルの名前を入力してください")
i = 0
for file_name in xlsx_df_list:
    i += 1
    df = pd.read_excel(f"{stats_path}/{file_name}.xlsx")
    merge_df = pd.concat([merge_df,df],axis=0,ignore_index=True)
    print(f"{i}/{len(xlsx_df_list)} {file_name}を結合しました")
merge_df.to_excel(f'{stats_path}/{merge_name}.xlsx', index=False)
print("全ファイル結合完了しました！")
